In [38]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from IPython.display import display
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors, KDTree
from scores import scores
import time
import warnings
from xgboost import XGBClassifier

warnings.filterwarnings("ignore")

### 4.) Import Data

In [39]:
%run ../notebooks/utils.ipynb

In [40]:
trainandknn_Xy_original_df = pd.read_csv("../data/train_new.csv", sep="|")
train_Xy_original_df, knn_Xy_original_df = train_test_split(trainandknn_Xy_original_df,train_size=0.75, random_state=42) # if FINAL_SUBMISSION else 0.8) #small
test_X_original_df  = pd.read_csv("../data/test.csv", sep="|").iloc[0:500] #TODO: For faster testing we use less data from the test set
test_final_X_df = pd.read_csv("../data/test.csv", sep="|")

train_Xy_wo_knn_df = pd.read_csv("../data/train_new.csv", sep="|")

#Only for test routines
val_Xy_original_df = pd.read_csv("../data/val_new.csv", sep="|")
train_complete_Xy_original_df = pd.read_csv("../data/train.csv", sep="|")

In [41]:
#convention for variables names: datasetname_columntype_transformstatus_dataframeornot
train_y_original_df = train_Xy_original_df[["fraud"]].copy()
train_X_original_df = train_Xy_original_df.copy().drop("fraud", axis=1)

knn_y_original_df = knn_Xy_original_df[["fraud"]].copy()
knn_X_original_df = knn_Xy_original_df.copy().drop("fraud", axis=1)

# Only for test routie#nes
val_y_originial_df = val_Xy_original_df[["fraud"]].copy()
val_X_originial_df = val_Xy_original_df.copy().drop("fraud", axis=1)

train_Xy_wo_knn_df = train_Xy_wo_knn_df.drop(train_Xy_wo_knn_df[train_Xy_wo_knn_df.trustLevel >= 3].index)

train_y_wo_knn_df = train_Xy_wo_knn_df[["fraud"]].copy()
train_X_wo_knn_df = train_Xy_wo_knn_df.copy().drop("fraud", axis=1)

train_complete_y_originial_df = train_complete_Xy_original_df[["fraud"]].copy()
train_complete_X_originial_df = train_complete_Xy_original_df.copy().drop("fraud", axis=1)

In [42]:
#scaler = MinMaxScaler()
scaler = StandardScaler()
transformer = DataTransformer(scaler)

# Adding new Features to train and test set
train_X_unscaled_df = transformer.add_features(train_X_original_df)
test_X_unscaled_df = transformer.add_features(test_X_original_df)
knn_X_unscaled_df = transformer.add_features(knn_X_original_df)

val_X_unscaled_df = transformer.add_features(val_X_originial_df)
train_complete_X_unscaled_df = transformer.add_features(train_complete_X_originial_df) 

transformer.fit_scaler(transformer.add_features(train_complete_X_unscaled_df.append(test_X_unscaled_df, sort=False)))
train_X_scaled_df = transformer.apply_scaler(train_X_unscaled_df)
knn_X_scaled_df   = transformer.apply_scaler(knn_X_unscaled_df)


test_X_scaled_df  = transformer.apply_scaler(test_X_unscaled_df)
val_X_scaled_df = transformer.apply_scaler(val_X_unscaled_df)
train_complete_X_scaled_df = transformer.apply_scaler(train_complete_X_unscaled_df)

train_X_wo_knn_unscaled_df = transformer.add_features(train_X_wo_knn_df.copy())
train_X_wo_knn_scaled_df = transformer.apply_scaler(train_X_wo_knn_unscaled_df)

# labels
train_y_df = train_y_original_df.copy()
val_y_df = val_y_originial_df.copy()
knn_y_df = knn_y_original_df.copy()

train_complete_X_scaled_df = transformer.apply_scaler(transformer.add_features(train_complete_Xy_original_df.copy().drop(columns=['fraud'])))
train_complete_y_df = train_complete_Xy_original_df.copy().fraud
train_complete_X_unscaled_df = transformer.add_features(train_complete_Xy_original_df.copy().drop(columns=['fraud']))

test_final_X_df = transformer.add_features(test_final_X_df)


train_complete_y_df = train_complete_y_df.drop(train_complete_X_unscaled_df[train_complete_X_unscaled_df.trustLevel >= 3].index)
train_complete_X_unscaled_df = train_complete_X_unscaled_df.drop(train_complete_X_unscaled_df[train_complete_X_unscaled_df.trustLevel >= 3].index)

display(train_complete_X_unscaled_df.head(5))
display(train_complete_y_df.head(5))

,trustLevel,totalScanTimeInSeconds,grandTotal,lineItemVoids,scansWithoutRegistration,quantityModifications,scannedLineItemsPerSecond,valuePerSecond,lineItemVoidsPerPosition,totalScannedLineItems
5,1,770,11.09,11,5,2,0.033766,0.014403,0.423077,26.0
7,2,1545,22.80,0,8,4,0.006472,0.014757,0.000000,10.0
9,2,725,41.08,10,2,4,0.037241,0.056662,0.370370,27.0
15,1,870,32.45,3,1,5,0.006897,0.037299,0.500000,6.0
23,2,125,25.50,5,6,2,0.192000,0.204000,0.208333,24.0


5     1
7     0
9     0
15    0
23    0
Name: fraud, dtype: int64

## Scoring functions
Defining multiple scores which should be tracked in the HyperParamSearch Object

In [43]:
score = scores.Scores()

In [44]:
scoring = {'AUC': 'roc_auc', 'FBeta': metrics.make_scorer(metrics.fbeta_score, beta=0.5172), "Precision":'precision', "Recall": 'recall', "AP": score.average_precision, "DMC" : score.dmc_score}

## Defining the paramteres which should be tuned
To tune the hyperparameters, i looked into the [documentation here](https://xgboost.readthedocs.io/en/latest/parameter.html#parameters-for-tree-booster). `randint` can be used for integer values, for float values, use `uniform`. 

You can also use a Grid search on single parameters to get a feeling for a good interval. If you want to try only two possibilities, you can create a list like for the `scale_pos_weight` parameter.

**Note: For the classifiers which work without gpu support, you can probably set a parameter n_jobs=-1 to use all processors**

In [46]:
params = {
    "max_depth": randint(2, 6),  # default 3
    "n_estimators": randint(200, 450),  # default 100
    "learning_rate" :  uniform(0.1, 0.7),
    "gamma" : uniform(0,2),
    "min_child_weight" : uniform(0,50),
    "max_delta_step" : uniform(0,10), #Set it to value of 1-10 might help control the update.
    "reg_lambda" : uniform(0.1,2),
}

## Creating a classifier with some default values
Not all paramters of a classifier should be fine tuned. For SVM for example, the `kernel`-paramter should be set manually. In the case of xgboost, some things like the objective, the booster and the tree method should not be tuned. The choice of paramters depend on the specific classifier

In [48]:
default_xgb = XGBClassifier(booster="gbtree",tree_method='gpu_hist', disable_default_eval_metric=1,objective='binary:logistic',eval_metric='aucpr', n_gpus=1, verbosity=2)
search = RandomizedSearchCV(default_xgb, scoring=scoring, param_distributions=params, random_state=42, n_iter=500,
                            cv=3, verbose=1, n_jobs=-1, return_train_score=True,refit='DMC')

In [49]:
search = RandomizedSearchCV(default_xgb, scoring=scoring, param_distributions=params, random_state=42, n_iter=2500,
                            cv=3, verbose=1, n_jobs=-1, return_train_score=True,refit='DMC')
search.fit(train_complete_X_unscaled_df, train_complete_y_df)
results = search.cv_results_

Fitting 3 folds for each of 2500 candidates, totalling 7500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    4.3s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:   17.8s
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed:   39.8s
[Parallel(n_jobs=-1)]: Done 776 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 1226 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 1776 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done 2426 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-1)]: Done 3176 tasks      | elapsed:  4.7min
[Parallel(n_jobs=-1)]: Done 4026 tasks      | elapsed:  5.9min
[Parallel(n_jobs=-1)]: Done 4976 tasks      | elapsed:  7.3min
[Parallel(n_jobs=-1)]: Done 6026 tasks      | elapsed:  8.7min
[Parallel(n_jobs=-1)]: Done 7176 tasks      | elapsed: 10.4min
[Parallel(n_jobs=-1)]: Done 7500 out of 7500 | elapsed: 10.8min finished


In [50]:
search.best_params_

{'gamma': 1.7116458142478073,
 'learning_rate': 0.576642048832076,
 'max_delta_step': 1.8657849925364134,
 'max_depth': 2,
 'min_child_weight': 1.8910068276017655,
 'n_estimators': 223,
 'reg_lambda': 1.0077372399918723}

In [51]:
train_complete_Xy_unscaled_df = train_complete_X_unscaled_df.copy()
train_complete_Xy_unscaled_df['fraud'] =train_complete_y_df.copy()

train_Xy_wo_knn_unscaled_df = train_X_wo_knn_unscaled_df.copy()
train_Xy_wo_knn_unscaled_df['fraud'] = train_y_wo_knn_df

val_Xy_unscaled = val_X_unscaled_df.copy()
val_Xy_unscaled['fraud'] = val_y_df.copy()

# need to cut trustLevel >= 3


In [52]:
train_complete_Xy_unscaled_df_lt = train_complete_Xy_unscaled_df[train_complete_Xy_unscaled_df.trustLevel <= 2]
train_Xy_wo_knn_unscaled_df_lt = train_Xy_wo_knn_unscaled_df[train_Xy_wo_knn_unscaled_df.trustLevel <=2]
val_Xy_unscaled_lt = val_Xy_unscaled[val_Xy_unscaled.trustLevel <= 2]

In [53]:
train_Xy_wo_knn_unscaled_df_lt.describe()

,trustLevel,totalScanTimeInSeconds,grandTotal,lineItemVoids,scansWithoutRegistration,quantityModifications,scannedLineItemsPerSecond,valuePerSecond,lineItemVoidsPerPosition,totalScannedLineItems,fraud
count,538.000000,538.000000,538.000000,538.000000,538.000000,538.000000,538.000000,538.000000,538.000000,538.000000,538.000000
mean,1.520446,916.704461,48.341004,5.565056,4.869888,2.485130,0.043364,0.164894,0.750969,15.265799,0.150558
std,0.500047,528.424724,29.504421,3.435011,3.170186,1.700247,0.115698,0.805122,1.273789,8.716836,0.357950
min,1.000000,2.000000,0.070000,0.000000,0.000000,0.000000,0.000551,0.000044,0.000000,1.000000,0.000000
25%,1.000000,463.250000,23.652500,3.000000,2.000000,1.000000,0.008607,0.024591,0.168478,8.000000,0.000000
50%,2.000000,906.500000,44.635000,5.000000,5.000000,2.000000,0.017055,0.051039,0.386752,15.000000,0.000000
75%,2.000000,1362.000000,75.317500,9.000000,8.000000,4.000000,0.031617,0.111131,0.800000,23.000000,0.000000
max,2.000000,1829.000000,99.820000,11.000000,10.000000,5.000000,1.611111,17.035000,10.000000,30.000000,1.000000


In [63]:
scorings = {'AUC': 'roc_auc', 'FBeta': metrics.make_scorer(metrics.fbeta_score, beta=0.5172), "Precision":'precision', "Recall": 'recall', "AP": score.average_precision, "DMC" : score.dmc_score}
# Alte eigene Suche (Schlechter)
xgbo = XGBClassifier(gamma = 0.5361394269637965,learning_rate=0.1,max_delta_step = 2.809035999095031,max_depth = 2, min_child_weight= 0.9672585190973582,n_estimators = 403,reg_lambda = 0.16439595189061076)
# Christians Suche
xgbo = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, disable_default_eval_metric=1,
       eval_metric='aucpr', gamma=0.5361394269637965,
       learnin_rate=0.5259503267995072, learning_rate=0.1,
       max_delta_step=2.809035999095031, max_depth=2,
       min_child_weight=0.9672585190973582, missing=None, n_estimators=403,
       n_gpus=1, n_jobs=1, nthread=None, objective='binary:logistic',
       random_state=0, reg_alpha=0, reg_lambda=0.16439595189061076,
       scale_pos_weight=1, seed=None, silent=True, subsample=1,
       tree_method='gpu_hist', verbosity=2)
result_dict = test_classification(xgbo,df_train=train_Xy_wo_knn_unscaled_df_lt, df_val=val_Xy_unscaled_lt)

Results Fix Split: 
DMC Score: 40  ---  Normalized DMC Score: 0.28368794326241137, 

Results Cross Validation: 
DMC Score: 51.0  ---  Normalized DMC Score: 0.37527233115468406 


In [60]:
dmc = np.mean(search.cv_results_['mean_test_DMC'])
ap = np.mean(search.cv_results_['mean_test_AP'])
precision = np.mean(search.cv_results_['mean_test_Precision'])
recall = np.mean(search.cv_results_['mean_test_Recall'])

In [61]:
print("DMC: {} --- Average Precision: {} --- Precision: {} --- Recall: {}".format(dmc, ap, precision, recall))

DMC: -90.17948748159057 --- Average Precision: 0.7031804942338902 --- Precision: 0.48450023241361406 --- Recall: 0.36629720325243503


In [62]:
search.best_estimator_

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, disable_default_eval_metric=1,
       eval_metric='aucpr', gamma=1.7116458142478073,
       learning_rate=0.576642048832076, max_delta_step=1.8657849925364134,
       max_depth=2, min_child_weight=1.8910068276017655, missing=None,
       n_estimators=223, n_gpus=1, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1.0077372399918723, scale_pos_weight=1, seed=None,
       silent=True, subsample=1, tree_method='gpu_hist', verbosity=2)

In [58]:
search.best_params_

{'gamma': 1.7116458142478073,
 'learning_rate': 0.576642048832076,
 'max_delta_step': 1.8657849925364134,
 'max_depth': 2,
 'min_child_weight': 1.8910068276017655,
 'n_estimators': 223,
 'reg_lambda': 1.0077372399918723}